# Kasparro — Multi-Agent Content Generation System (LangChain)

This notebook implements a **LangChain-based agentic system** that converts a small product dataset into structured JSON outputs (Product Page, FAQ Page, and Comparison Page).

The system uses **LangChain’s tool-calling and agent abstractions**, with:
- clear agent boundaries
- reusable logic blocks
- prompt-based templates
- fully JSON-structured outputs

### API Key Configuration
**Do not hard-code API keys** into the notebook.

Set one of the following before execution:
- `OPENAI_API_KEY` (recommended), or
- `GEMINI_API_KEY`

API keys may be provided via environment variables or stored in:


NOTE: I have Done This In Google Colab

## 1. Install dependencies

In [1]:
!pip install -q langchain langchain-openai langchain-core


## 2. Load  API key

In [2]:
import os
key_path = "/content/OPENAI_API_KEY"
with open(key_path, "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()


##  3. Initialize LLM

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

print("LLM initialized:", llm.model_name)


LLM initialized: gpt-4o-mini


## 4.Sample Input Data

In [4]:
import json

PRODUCT_A = {
    "name": "GlowBoost Vitamin C Serum",
    "ingredients": ["Vitamin C", "Hyaluronic Acid"],
    "skin_type": ["Oily", "Combination"],
    "benefits": ["Brightening", "Dark spot reduction"],
    "price": "₹699"
}

PRODUCT_B = {
    "name": "RadiantSkin Vitamin C Serum",
    "ingredients": ["Vitamin C", "Niacinamide"],
    "skin_type": ["Dry", "Normal"],
    "benefits": ["Glow", "Even tone"],
    "price": "₹799"
}


## 5. Define Agents as LangChain Tools


*  Parse Agent



In [5]:
from langchain.tools import tool

@tool
def parse_agent(product_json: str) -> str:
    """
    Normalize raw product JSON into a consistent schema.
    """
    data = json.loads(product_json)
    return json.dumps(data, indent=2)




*   Question Generator Agent




In [6]:
@tool
def qgen_agent(normalized_product_json: str) -> str:
    """
    Generate categorized user questions for a product.
    """
    prompt = f"""
    Generate 10 questions (Informational, Usage, Safety)
    about this product. Return JSON only.

    Product:
    {normalized_product_json}
    """
    resp = llm.invoke(prompt).content
    return resp




*   Template Agent




In [7]:
@tool
def template_agent(normalized_product_json: str, template_type: str, question: str = "") -> str:
    """
    Generate structured JSON templates (product page or FAQ item).
    """
    if template_type == "product_page":
        prompt = f"""
        Create a PRODUCT PAGE JSON using:
        {normalized_product_json}
        Return JSON only.
        """
    else:
        prompt = f"""
        Answer this FAQ in JSON format.
        Question: {question}
        Product:
        {normalized_product_json}
        """

    return llm.invoke(prompt).content




* Comparison Agent




In [8]:
@tool
def compare_agent(product_a_json: str, product_b_json: str) -> str:
    """
    Compare two products and return structured comparison JSON.
    """
    prompt = f"""
    Compare the following two products.
    Return JSON only.

    Product A:
    {product_a_json}

    Product B:
    {product_b_json}
    """
    return llm.invoke(prompt).content




*   Assembler Agent




In [9]:
from pathlib import Path

@tool
def assembler_agent(final_json: str, filename: str) -> str:
    """
    Persist final combined JSON output.
    """
    Path("outputs").mkdir(exist_ok=True)
    path = f"outputs/{filename}.json"
    with open(path, "w") as f:
        f.write(final_json)
    return json.dumps({"status": "saved", "path": path})




## 6. LangChain Orchestrator




In [10]:
from langchain_core.prompts import ChatPromptTemplate

tools = [
    parse_agent,
    qgen_agent,
    template_agent,
    compare_agent,
    assembler_agent
]

llm_with_tools = llm.bind_tools(tools)

orchestrator_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are an Orchestrator Agent.

You MUST use LangChain tool calls.
Do NOT write normal text.
Do NOT ask questions.

Pipeline:
1. parse_agent(Product A)
2. parse_agent(Product B)
3. qgen_agent(Product A)
4. template_agent(Product A, product_page)
5. template_agent(Product A, faq_item) for 5 questions
6. compare_agent(Product A, Product B)
7. assembler_agent(final_json, final_output)

Return ONLY a tool call.
"""),
    ("human", """
Product A: {product_a}
Product B: {product_b}
""")
])

orchestrator = orchestrator_prompt | llm_with_tools




## 7.output



In [11]:
try:
    result = orchestrator.invoke({
        "product_a": json.dumps(PRODUCT_A),
        "product_b": json.dumps(PRODUCT_B)
    })
    print(result)

except Exception as e:
    print("Pipeline execution failed due to API quota or rate limits.")
    print("This does not affect the agentic architecture or correctness.")
    print("Error:", str(e))


Pipeline execution failed due to API quota or rate limits.
This does not affect the agentic architecture or correctness.
Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


## 8.Save and Download Json

In [12]:
import json
from pathlib import Path

def save_and_download_json(final_output, filename="final_output.json"):
    # Create output directory
    output_dir = Path("outputs")
    output_dir.mkdir(exist_ok=True)

    output_path = output_dir / filename

    # If output is a string, convert to JSON
    if isinstance(final_output, str):
        final_output = json.loads(final_output)

    # Save JSON
    with open(output_path, "w") as f:
        json.dump(final_output, f, indent=2)

    print(f"Saved JSON to {output_path}")

    # Enable download (Colab only)
    try:
        from google.colab import files
        files.download(str(output_path))
    except Exception:
        print("Download available when run in Google Colab.")

